Data preprocessing:

1. remove the small classes which has the size smaller than 50
2. shuffle the class order 3 times to generate 3 splits for the experiment

In [1]:
import os
import scipy
import torch
import scipy.io
import scipy.io
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from torch_geometric.data import Data
from torch_geometric.datasets import Yelp
from utils import sparse_mx_to_torch_sparse_tensor
from sklearn.model_selection import train_test_split

In [185]:
#function sparse_mx_to_torch_sparse_tensor
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [2]:
#load dataset: Blogcatalog
data_name = "blogcatalog"
path = "data/blogcatalog/"
print('Loading dataset ' + data_name + '.mat...')
mat = scipy.io.loadmat(path + data_name)
labels = mat['group']
blog_labels = sparse_mx_to_torch_sparse_tensor(labels).to_dense()



Loading dataset blogcatalog.mat...


In [3]:
blog_lbl_clean = torch.load(path+"blog_clean_lbl.pt")
cls = torch.transpose(blog_lbl_clean, 0, 1)
print(cls.shape)
cls_asgn = {}
for i, label in enumerate(cls):
    cls_asgn[i] = torch.nonzero(label).flatten()

torch.Size([36, 10312])


In [4]:
size = []
for i, c in enumerate(list(cls_asgn.keys())):
    size.append(len(cls_asgn[c]))

cls_order = sorted(range(len(size)), key=lambda i: size[i])
cls_order

[35,
 33,
 32,
 13,
 25,
 31,
 29,
 26,
 34,
 0,
 27,
 23,
 3,
 19,
 24,
 20,
 12,
 16,
 18,
 8,
 21,
 28,
 30,
 9,
 14,
 15,
 10,
 11,
 6,
 2,
 1,
 5,
 4,
 22,
 17,
 7]

In [8]:
sorted(size)

[53,
 62,
 64,
 91,
 94,
 95,
 98,
 112,
 137,
 167,
 173,
 201,
 246,
 246,
 284,
 287,
 306,
 311,
 320,
 323,
 325,
 372,
 386,
 405,
 415,
 467,
 472,
 514,
 610,
 619,
 778,
 876,
 908,
 977,
 986,
 1623]

In [187]:
#process blogcatlog
l = torch.transpose(blog_labels, 0, 1)
cls_size = [torch.nonzero(ll).shape[0] for ll in l]
rmv_cls=[cls_size.index(s) for s in cls_size if s<50]

kp_cls = [i for i in range(blog_labels.shape[1]) if i not in rmv_cls]
blog_lbl_clean = blog_labels[:, kp_cls]
torch.save(blog_lbl_clean, 'data/blogcatalog/blog_clean_lbl.pt')

In [273]:
rmv_cls=[cls_size.index(s) for s in cls_size if s<50]
print(rmv_cls)

[11, 37, 38]


In [188]:
# the order to present in Class-IL setting
from sklearn.utils import shuffle
cls_order = shuffle(range(blog_lbl_clean.shape[1]))
print(cls_order)
torch.save(cls_order, 'data/blogcatalog/blog_shuffle1.pt')
cls_order = shuffle(range(blog_lbl_clean.shape[1]))
print(cls_order)
torch.save(cls_order, 'data/blogcatalog/blog_shuffle2.pt')
cls_order = shuffle(range(blog_lbl_clean.shape[1]))
torch.save(cls_order, 'data/blogcatalog/blog_shuffle3.pt')
print(cls_order)

[18, 19, 16, 24, 1, 4, 6, 7, 8, 20, 35, 31, 29, 28, 2, 11, 25, 13, 5, 32, 26, 34, 15, 10, 23, 14, 3, 33, 17, 30, 22, 0, 21, 9, 27, 12]
[9, 8, 14, 3, 25, 11, 18, 20, 16, 31, 26, 2, 22, 0, 35, 32, 5, 24, 23, 33, 15, 6, 29, 4, 12, 28, 13, 17, 7, 10, 30, 19, 34, 21, 27, 1]
[4, 9, 12, 17, 35, 26, 28, 0, 2, 1, 10, 8, 33, 32, 20, 34, 3, 15, 6, 14, 19, 29, 21, 5, 11, 25, 23, 31, 13, 7, 24, 27, 22, 30, 16, 18]


In [189]:
n_cls = blog_lbl_clean.shape[1]
print(n_cls)

36


In [190]:
cls_order1 = torch.load(path + "blog_shuffle1.pt")
cls_order2 = torch.load(path + "blog_shuffle2.pt")
cls_order3 = torch.load(path + "blog_shuffle3.pt")
print(cls_order1)
print(cls_order2)
print(cls_order3)

[18, 19, 16, 24, 1, 4, 6, 7, 8, 20, 35, 31, 29, 28, 2, 11, 25, 13, 5, 32, 26, 34, 15, 10, 23, 14, 3, 33, 17, 30, 22, 0, 21, 9, 27, 12]
[9, 8, 14, 3, 25, 11, 18, 20, 16, 31, 26, 2, 22, 0, 35, 32, 5, 24, 23, 33, 15, 6, 29, 4, 12, 28, 13, 17, 7, 10, 30, 19, 34, 21, 27, 1]
[4, 9, 12, 17, 35, 26, 28, 0, 2, 1, 10, 8, 33, 32, 20, 34, 3, 15, 6, 14, 19, 29, 21, 5, 11, 25, 23, 31, 13, 7, 24, 27, 22, 30, 16, 18]


In [191]:
n_cls_per_t = 2
groups_idx1 = [tuple(cls_order1[i:i+n_cls_per_t]) for i in range(0, n_cls-1, n_cls_per_t)]
groups_idx2 = [tuple(cls_order2[i:i+n_cls_per_t]) for i in range(0, n_cls-1, n_cls_per_t)]
groups_idx3 = [tuple(cls_order3[i:i+n_cls_per_t]) for i in range(0, n_cls-1, n_cls_per_t)]
groups_idxs = []
groups_idxs.append(groups_idx1)
groups_idxs.append(groups_idx2)
groups_idxs.append(groups_idx3)

In [192]:
cls = torch.transpose(blog_lbl_clean, 0, 1)
print(cls.shape)
cls_asgn = {}
for i, label in enumerate(cls):
    cls_asgn[i] = torch.nonzero(label).flatten()
cls_asgn.keys()

torch.Size([36, 10312])


dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])

In [193]:
# TaskIL split
# requirment: no duplicates within each group of classes
import random

#split: 60%train, 10%val, 30%test
train_ratio = 0.6
# 25% of the rest of the nodes
val_ratio = 0.25


# three orders to present the classes
for k, groups_idx in enumerate(groups_idxs):
    
    # nested dictionary splits={group1:{train:[], val:[], test[]}, group2...}
    splits={}
    
    # each group of classes in one order
    for group in groups_idx:
        
        print("##########################################")
        print("classes in this group", group)
        train_ids = []
        val_ids = []
        test_ids = []
        split={}
        # check for duplicates: nodes ids already seen, no duolicates in n_g
        n_g = []
        # nodes ids for the duplicates
        dup = []
        for i, c in enumerate(group):
            
            # the first class: 60%, 10%, 30%
            if i == 0:
                n_c = cls_asgn[c].tolist()
                n_g.extend(n_c)
                ids_train, ids_val_test = train_test_split(n_c, test_size=0.4, random_state=40)
                ids_val, ids_test = train_test_split(ids_val_test, test_size=0.75, random_state=39)
                train_ids.extend(ids_train)
                val_ids.extend(ids_val)
                test_ids.extend(ids_test)    
                print("number of nodes in total in c1",len(n_c))
                print("total train for c1",len(ids_train))
                print("total val for c1",len(ids_val))
                print("total test for c1",len(ids_test))
            # other classes in the group
            else:
                print(c)
                #check for duplicate nodes in the previous classes and the current class
                n_c = cls_asgn[c].tolist()
                n_c_all = n_c.copy()
                print("number of nodes in total in c2",len(n_c))
                for m in n_c_all:
                    # if there is, remove these nodes from random split
                    if m in n_g:
                        n_c.remove(m)
                        dup.append(m)
                        print("found duplicates: node ", m)
                print(len(n_c))
                print("check for duplicates in second class", len(set(n_c)))
                n_g.extend(n_c)
                print("after removal number of nodes in current class", len(n_c))
                
                #split current class
                # num_n in train, val, test in total to sample for this class
                n_c_train = int(len(n_c_all) * train_ratio)
                n_c_val = int((len(n_c_all) - n_c_train) * 0.25)
                n_c_test = len(n_c_all) - n_c_train - n_c_val
                print("total train for c2",n_c_train)
                print("total val for c2",n_c_val)
                print("total test for c2",n_c_test)

                # number to sample from the new apeared nodes
                for d in dup:
                    if d in train_ids:
                        n_c_train = n_c_train - 1
                    if d in val_ids:
                        n_c_val = n_c_val - 1
                    if d in test_ids:
                        n_c_test = n_c_test - 1
                # sample training nodes
                print("number of nodes after removal",len(n_c))
                print("remain in train set to sample",n_c_train)
                print("remain in val set to sample",n_c_val)
                print("remain in test set to sample",n_c_test)

                train_to_sample = random.sample(n_c, k=n_c_train)
                train_ids.extend(train_to_sample)
                print(len(train_to_sample))
                remain_nodes = list(set(n_c) - set(train_to_sample))
                print("remain_nodes", len(remain_nodes))
                #sample val nodes
                val_to_sample = random.sample(remain_nodes, k=n_c_val)
                print(len(val_to_sample))
                val_ids.extend(val_to_sample)
                #sample test nodes
                remain_nodes = list(set(remain_nodes) - set(val_to_sample))
                print("remain_nodes", len(remain_nodes))
                print(len(remain_nodes))
                #remains go to test set
                test_ids.extend(remain_nodes)

                 
        # check for duplicates
        print(len(n_g))
        print("all nodes at t",len(list(set(n_g))))
        print("final number of training nodes",len(train_ids))
        print("final number of val nodes",len(val_ids))
        print("final number of test nodes",len(test_ids))
        for i in train_ids:
            if i in val_ids:
                print(i)
                raise Exception("Sorry, train found in val")
            if i in test_ids:
                print(i)
                raise Exception("Sorry, train found in test")
        for i in val_ids:
            if i in test_ids:
                print(i)
                raise Exception("Sorry, val found in test")
        
        # save in split
        split["train"] = train_ids
        split["val"] = val_ids
        split["test"] = test_ids
        splits[group] = split
        torch.save(splits, "data/blogcatalog/split_"+str(k)+".pt")

##########################################
classes in this group (18, 19)
number of nodes in total in c1 320
total train for c1 192
total val for c1 32
total test for c1 96
19
number of nodes in total in c2 246
found duplicates: node  8716
245
check for duplicates in second class 245
after removal number of nodes in current class 245
total train for c2 147
total val for c2 24
total test for c2 75
number of nodes after removal 245
remain in train set to sample 147
remain in val set to sample 24
remain in test set to sample 74
147
remain_nodes 98
24
remain_nodes 74
74
565
all nodes at t 565
final number of training nodes 339
final number of val nodes 56
final number of test nodes 170
##########################################
classes in this group (16, 24)
number of nodes in total in c1 311
total train for c1 186
total val for c1 31
total test for c1 94
24
number of nodes in total in c2 284
found duplicates: node  189
found duplicates: node  837
found duplicates: node  928
found duplicat

##########################################
classes in this group (27, 1)
number of nodes in total in c1 173
total train for c1 103
total val for c1 17
total test for c1 53
1
number of nodes in total in c2 778
found duplicates: node  1096
found duplicates: node  5914
found duplicates: node  7236
775
check for duplicates in second class 775
after removal number of nodes in current class 775
total train for c2 466
total val for c2 78
total test for c2 234
number of nodes after removal 775
remain in train set to sample 465
remain in val set to sample 78
remain in test set to sample 232
465
remain_nodes 310
78
remain_nodes 232
232
948
all nodes at t 948
final number of training nodes 568
final number of val nodes 95
final number of test nodes 285
##########################################
classes in this group (4, 9)
number of nodes in total in c1 908
total train for c1 544
total val for c1 91
total test for c1 273
9
number of nodes in total in c2 405
found duplicates: node  218
found dupli

In [198]:
import torch
splits = torch.load("data/blogcatalog/split_0.pt")


In [205]:
print(len(splits[(8,20)]["train"]))
print(len(splits[(8,20)]["val"]))
print(len(splits[(8,20)]["test"]))

364
60
185


In [207]:
for i in splits[(8,20)]["val"]:
    if i in splits[(8,20)]["train"]:
        print("train in val")
    if i in splits[(8,20)]["test"]:
        print("train in test")

In [210]:
data_name = "blogcatalog"
path = "data/blogcatalog/"
print('Loading dataset ' + data_name + '.mat...')
mat = scipy.io.loadmat(path + data_name)

Loading dataset blogcatalog.mat...


In [211]:
labels = torch.load(path+"blog_clean_lbl.pt")
print(labels.shape)


torch.Size([10312, 36])


In [212]:
adj = mat['network']
adj = sparse_mx_to_torch_sparse_tensor(adj).to_dense()
edge_index = torch.transpose(torch.nonzero(adj), 0, 1).long()
features = torch.eye(labels.shape[0]).float()

In [216]:
print(adj.shape)
print(edge_index.shape)
print(features.shape)

torch.Size([10312, 10312])
torch.Size([2, 667966])
torch.Size([10312, 10312])


In [235]:
shuffle_idx = "shuffle3"

groups = torch.load(path + "blog_" + shuffle_idx +".pt")
splits =  torch.load(path + "split_" + shuffle_idx[-1] + ".pt")

In [236]:
splits

{(4,
  9): {'train': [1005,
   5208,
   9578,
   15,
   4334,
   1731,
   4370,
   5050,
   6017,
   2374,
   2031,
   400,
   1680,
   8614,
   1623,
   5221,
   861,
   1485,
   6489,
   1674,
   7980,
   3921,
   3658,
   1489,
   6791,
   4339,
   7207,
   6143,
   2736,
   7466,
   496,
   10154,
   2477,
   485,
   2601,
   3182,
   3235,
   4315,
   9223,
   1023,
   2225,
   517,
   3119,
   74,
   6136,
   7322,
   2322,
   7739,
   6834,
   4712,
   8300,
   2012,
   1899,
   1643,
   8030,
   2592,
   7525,
   7192,
   3325,
   4428,
   3173,
   2431,
   8845,
   3460,
   2976,
   5303,
   795,
   862,
   8395,
   8677,
   1647,
   1050,
   1479,
   1404,
   4198,
   2357,
   1882,
   3544,
   4499,
   6046,
   6401,
   6316,
   7134,
   469,
   482,
   100,
   1033,
   3632,
   3461,
   687,
   7802,
   5015,
   5416,
   4623,
   6250,
   8201,
   8405,
   9537,
   1012,
   1271,
   6433,
   1932,
   9197,
   1700,
   1971,
   4399,
   6471,
   8367,
   9285,
   1994,
   10

In [237]:
groups

[4,
 9,
 12,
 17,
 35,
 26,
 28,
 0,
 2,
 1,
 10,
 8,
 33,
 32,
 20,
 34,
 3,
 15,
 6,
 14,
 19,
 29,
 21,
 5,
 11,
 25,
 23,
 31,
 13,
 7,
 24,
 27,
 22,
 30,
 16,
 18]

In [263]:
for i in range(3):
    splits = torch.load(path + "split_" + str(i+1) + ".pt")
    groups = {}
    for sub_g in splits.keys():
        ids = []
        print(sub_g)
        for split in splits[sub_g].keys():
            print(split)
            dct = splits[sub_g]
            ids.extend(dct[split])
        groups[sub_g] = ids
      
    torch.save(groups, "data/blogcatalog/groups" + str(i+1) + ".pt")

(18, 19)
train
val
test
(16, 24)
train
val
test
(1, 4)
train
val
test
(6, 7)
train
val
test
(8, 20)
train
val
test
(35, 31)
train
val
test
(29, 28)
train
val
test
(2, 11)
train
val
test
(25, 13)
train
val
test
(5, 32)
train
val
test
(26, 34)
train
val
test
(15, 10)
train
val
test
(23, 14)
train
val
test
(3, 33)
train
val
test
(17, 30)
train
val
test
(22, 0)
train
val
test
(21, 9)
train
val
test
(27, 12)
train
val
test
(9, 8)
train
val
test
(14, 3)
train
val
test
(25, 11)
train
val
test
(18, 20)
train
val
test
(16, 31)
train
val
test
(26, 2)
train
val
test
(22, 0)
train
val
test
(35, 32)
train
val
test
(5, 24)
train
val
test
(23, 33)
train
val
test
(15, 6)
train
val
test
(29, 4)
train
val
test
(12, 28)
train
val
test
(13, 17)
train
val
test
(7, 10)
train
val
test
(30, 19)
train
val
test
(34, 21)
train
val
test
(27, 1)
train
val
test
(4, 9)
train
val
test
(12, 17)
train
val
test
(35, 26)
train
val
test
(28, 0)
train
val
test
(2, 1)
train
val
test
(10, 8)
train
val
test
(33, 32)
train
val

In [272]:
for i in range(3):
    groups = torch.load("data/blogcatalog/groups" + str(i+1) + ".pt")
    splits = torch.load("data/blogcatalog/split_" + str(i+1) + ".pt")
    ids=[]
    for sub_g in splits.keys():
        for split in splits[sub_g].keys():
            ids.extend(splits[sub_g][split])
        ids_g = groups[sub_g]
        print(ids_g)
        if ids_g == ids:
            print("success!")
    break

[3067, 1957, 5987, 3146, 2425, 8009, 3570, 6655, 5299, 9110, 5202, 9339, 5318, 2269, 7494, 8099, 5113, 4218, 5034, 5478, 2189, 2085, 4609, 198, 2678, 5374, 1997, 1766, 2531, 2068, 2636, 620, 9349, 6111, 7744, 7979, 8366, 1047, 1342, 7845, 7257, 3453, 6336, 5073, 6071, 2709, 5356, 1849, 1006, 7124, 1240, 5226, 6368, 4163, 4270, 623, 4995, 3513, 2995, 8793, 3979, 3749, 8458, 1807, 7044, 3859, 2907, 270, 506, 9340, 7599, 1857, 6478, 9185, 8709, 8975, 9158, 4364, 1604, 7009, 5020, 3506, 2919, 5241, 6304, 2433, 4257, 9691, 5395, 3258, 8510, 5649, 4302, 1726, 3981, 10224, 1140, 4697, 2100, 7467, 3522, 5054, 1226, 9309, 2179, 6633, 2918, 1051, 562, 1290, 1296, 867, 2812, 8562, 2649, 583, 3609, 686, 3230, 3318, 5778, 6236, 443, 9947, 2285, 2356, 6137, 5827, 5337, 5866, 7215, 892, 10078, 2033, 171, 9379, 1762, 6855, 937, 2339, 8146, 1493, 2720, 9537, 6037, 4573, 2998, 3270, 7000, 6207, 6813, 2667, 2729, 6301, 10103, 2190, 2962, 9169, 802, 6412, 1705, 787, 9751, 5996, 7025, 9965, 5769, 10300, 37

In [269]:
path = "data/yelp/"
for i in range(3):
    splits = torch.load(path + "split_" + str(i+1) + ".pt")
    groups = {}
    for sub_g in splits.keys():
        ids = []
        print(sub_g)
        for split in splits[sub_g].keys():
            print(split)
            dct = splits[sub_g]
            ids.extend(dct[split])
        groups[sub_g] = ids
      
    torch.save(groups, "data/yelp/groups" + str(i+1) + ".pt")

(76, 13)
train
val
test
(84, 65)
train
val
test
(23, 0)
train
val
test
(46, 78)
train
val
test
(37, 42)
train
val
test
(52, 10)
train
val
test
(59, 96)
train
val
test
(40, 91)
train
val
test
(16, 73)
train
val
test
(93, 32)
train
val
test
(29, 50)
train
val
test
(77, 6)
train
val
test
(30, 53)
train
val
test
(7, 9)
train
val
test
(54, 2)
train
val
test
(8, 86)
train
val
test
(48, 55)
train
val
test
(15, 81)
train
val
test
(58, 24)
train
val
test
(89, 26)
train
val
test
(20, 28)
train
val
test
(71, 95)
train
val
test
(11, 61)
train
val
test
(57, 31)
train
val
test
(19, 45)
train
val
test
(98, 17)
train
val
test
(33, 69)
train
val
test
(62, 60)
train
val
test
(64, 5)
train
val
test
(22, 27)
train
val
test
(25, 79)
train
val
test
(72, 49)
train
val
test
(41, 34)
train
val
test
(36, 21)
train
val
test
(38, 56)
train
val
test
(3, 67)
train
val
test
(12, 94)
train
val
test
(14, 43)
train
val
test
(51, 1)
train
val
test
(75, 83)
train
val
test
(85, 99)
train
val
test
(66, 47)
train
val
test
(

In [277]:
col = [18,19,16,24]
blog_lbl_clean[:,col]

tensor([[0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [311]:
splits = torch.load("data/blogcatalog/split_1.pt")
groups = torch.load("data/blogcatalog/groups1.pt")
ids = groups[(16, 24)]
l = blog_lbl_clean[ids]
l_t = torch.transpose(l, 0, 1)

d1 = torch.nonzero(l_t[18, :]).flatten().tolist()
l18_id = torch.tensor(ids)[d1]

d2 = torch.nonzero(l_t[19, :]).flatten().tolist()
l19_id = torch.tensor(ids)[d2]
print(l18_id)
print(l19_id)

tensor([ 514, 5785, 7124, 6111,  686,  620, 9965, 6400, 5478, 6304, 6655, 3525])
tensor([ 734, 7226, 8426, 9670])


In [312]:
d1

[2, 91, 119, 214, 241, 273, 294, 301, 306, 338, 407, 572]

In [305]:
blog_lbl_clean[514]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [306]:
for n in l18_id:
    if n in splits[(16,24)]["train"]:
        print("node", n ,"in train")
    if n in splits[(16,24)]["val"]:
        print("node", n ,"in val")
    if n in splits[(16,24)]["test"]:
        print("node", n ,"in test")

node tensor(514) in train
node tensor(5785) in train
node tensor(7124) in train
node tensor(6111) in train
node tensor(686) in train
node tensor(620) in train
node tensor(9965) in train
node tensor(6400) in train
node tensor(5478) in train
node tensor(6304) in train
node tensor(6655) in test
node tensor(3525) in test


In [307]:
for n in l19_id:
    if n in splits[(16,24)]["train"]:
        print("node", n ,"in train")
    if n in splits[(16,24)]["val"]:
        print("node", n ,"in val")
    if n in splits[(16,24)]["test"]:
        print("node", n ,"in test")

node tensor(734) in train
node tensor(7226) in train
node tensor(8426) in train
node tensor(9670) in test


In [10]:
path = "data/dblp/"
data_name = "dblp"
print('Loading dataset DBLP...')
features = torch.FloatTensor(np.genfromtxt(os.path.join(path, "features.txt"), delimiter=",", dtype=np.float64))
labels = torch.FloatTensor(np.genfromtxt(os.path.join(path, "labels.txt"), delimiter=","))
cls = torch.transpose(labels, 0, 1)
cls_asgn = {}
for i, label in enumerate(cls):
    cls_asgn[i] = torch.nonzero(label).flatten()

Loading dataset DBLP...


In [11]:
cls_asgn

{0: tensor([    0,     1,     2,  ..., 28689, 28693, 28701]),
 1: tensor([    0,     1,     2,  ..., 28692, 28696, 28697]),
 2: tensor([    0,     1,     2,  ..., 28691, 28694, 28698]),
 3: tensor([    0,     1,     2,  ..., 28695, 28699, 28700])}

In [12]:
size = []
for i, c in enumerate(list(cls_asgn.keys())):
    size.append(len(cls_asgn[c]))

In [13]:
size

[8395, 5794, 12534, 7149]

In [16]:
path = "data/"
data_name = "pcg_removed_isolated_nodes"

print('Loading dataset ' + data_name + '.csv...')

labels = np.genfromtxt(os.path.join(path, data_name, "labels.csv"),
                       dtype=np.dtype(float), delimiter=',')
labels = torch.tensor(labels).float()
cls = torch.transpose(labels, 0, 1)
cls_asgn = {}
for i, label in enumerate(cls):
    cls_asgn[i] = torch.nonzero(label).flatten()
size = []
for i, c in enumerate(list(cls_asgn.keys())):
    size.append(len(cls_asgn[c]))
size

Loading dataset pcg_removed_isolated_nodes.csv...


[1018, 400, 167, 219, 634, 128, 331, 287, 650, 467, 213, 219, 959, 367, 175]